In [4]:
from ase.io import read
from ase.data import atomic_numbers
import numpy as np

# Read CIF
atoms = read("/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed.cif")

# Define valence charges manually or from a dict
valence_lookup = {
    'La': 11,  # adjust as per UPF
    'Zr': 4,
    'O': 6,
    'Li': 1,
}

# Write fractional coordinates
with open('/home/phanim/harshitrawat/summer/OOD_test/coordinates.inp', 'w') as f:
    for atom in atoms:
        symbol = atom.symbol
        Z = atomic_numbers[symbol]
        valence = valence_lookup[symbol]
        frac = atoms.get_scaled_positions()[atom.index]
        f.write(f"{Z} {valence} {frac[0]:.8f} {frac[1]:.8f} {frac[2]:.8f}\n")


In [5]:
cell = atoms.cell.array  # 3x3 matrix
bohr = 1.8897259886
with open('/home/phanim/harshitrawat/summer/OOD_test/domainVectors.inp', 'w') as f:
    for vec in cell:
        f.write(f"{vec[0]*bohr:.8f} {vec[1]*bohr:.8f} {vec[2]*bohr:.8f}\n")


In [8]:
from collections import defaultdict
import math

# Load coordinates.inp
with open("/home/phanim/harshitrawat/summer/OOD_test/coordinates.inp", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

natoms = len(lines)
species_set = set()
valence_total = 0
species_count = defaultdict(int)

for line in lines:
    tokens = line.split()
    if len(tokens) < 5:
        continue
    atomic_number = int(tokens[0])
    valence = float(tokens[1])
    
    species_set.add(atomic_number)
    valence_total += valence
    species_count[atomic_number] += 1

natom_types = len(species_set)
ks_wavefunctions = math.ceil(1.2 * valence_total / 2)

print("🧪 DFT-FE slab summary:")
print(f"  - Total atoms (NATOMS): {natoms}")
print(f"  - Unique atom types (NATOM TYPES): {natom_types}")
print(f"  - Species breakdown: {dict(species_count)}")
print(f"  - Total valence electrons: {valence_total}")
print(f"  - Recommended NUMBER OF KOHN-SHAM WAVEFUNCTIONS: {ks_wavefunctions}")


🧪 DFT-FE slab summary:
  - Total atoms (NATOMS): 648
  - Unique atom types (NATOM TYPES): 4
  - Species breakdown: {3: 444, 57: 36, 40: 24, 8: 144}
  - Total valence electrons: 1800.0
  - Recommended NUMBER OF KOHN-SHAM WAVEFUNCTIONS: 1080


In [1]:
from ase.io import read

atoms = read("/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed.cif")
cell = atoms.get_cell()

# Heuristic: check if vacuum exists along any direction
lengths = cell.lengths()
angles = cell.angles()

# Check vacuum-like behavior (e.g., >30 Å) to detect non-periodic axis
for i, L in enumerate(lengths):
    axis = ['x', 'y', 'z'][i]
    if L > 30:
        print(f"Likely non-periodic in {axis} (vacuum slab present)")
    else:
        print(f"Likely periodic in {axis}")


Likely periodic in x
Likely periodic in y
Likely non-periodic in z (vacuum slab present)
